workflow:

Load the ResNet model from models.py.

Extract frames from videos at a specified rate.

Use the model to predict labels for these frames.

Map the predictions to the labels provided in your text files (category_momentsv2.txt and category_multi_momentsv2.txt).

Compile the results into a CSV file.

1.Download and Run the model.py first (from github)

2. import model class (resnet50), load model and process video

In [27]:
import os
os.chdir('/Users/wangjiji/Desktop/moments_models-2')  # Replace with the path to your models.py
from models import resnet50
import cv2
import torch
import torchvision.transforms as transforms
import pandas as pd


# Load the model
model = resnet50(pretrained=True)  # Ensure this matches your specific function to load resnet50
model.eval()

# Define the transformations
transform = transforms.Compose([
    transforms.ToPILImage(),                # Convert numpy array to PIL Image
    transforms.Resize((224, 224)),          # Resize the image to 224x224
    transforms.ToTensor(),                  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Label mapping from your text file (adjust path as necessary)
with open('category_momentsv2.txt', 'r') as f:
    labels = {i: label.strip() for i, label in enumerate(f.readlines())}

# Directory containing the videos
directory = '/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/'

# Prepare to collect results
results = []

# Process each video file from 001 to 071
for i in range(1, 72):  # Adjust range for your video numbers
    video_number = f"{i:03}"
    video_path = f"{directory}{video_number}.mp4"
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Skipping: Could not open {video_path}")
        continue  # Skip this video file

    frame_rate = 16
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = max(int(frame_count / frame_rate), 1)  # Prevents zero division

    for frame_id in range(0, frame_count, interval):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        ret, frame = cap.read()
        if not ret:
            continue
        
        # Apply transformations and predict
        tensor = transform(frame)
        with torch.no_grad():
            outputs = model(tensor.unsqueeze(0))
            predicted_label = labels[torch.argmax(outputs).item()]

        results.append([video_number, frame_id, predicted_label])

    cap.release()

# Save results to CSV
# Creating the initial DataFrame
df = pd.DataFrame(results, columns=['Video', 'Frame', 'Label'])

# Pivoting the DataFrame
pivot_df = df.pivot(index='Video', columns='Frame', values='Label')

# Saving the pivoted DataFrame to CSV
pivot_df.to_csv('/Users/wangjiji/Desktop/video_labels.csv')


OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/006.mp4"


Skipping: Could not open /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/006.mp4


OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/009.mp4"


Skipping: Could not open /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/009.mp4


OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/021.mp4"


...

Skipping: Could not open /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/046.mp4


OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/053.mp4"


Skipping: Could not open /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/053.mp4


OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/058.mp4"


Skipping: Could not open /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/058.mp4
